# Umbrella Hook v4

## Key Facts

* Beam is 16mm w x 24mm tall at 12.5º angle
* Wing overhang 12mm-> 36mm

| Dimension   | value |
| ----------- | ----- |
| hook_length = 48
| beam_width  = 16
| beam_height = 24
| beam_angle  = 12.5
| drop        = hook_length*tan(beam_angle)


In [36]:
from math import sin,cos,tan, asin,acos, atan, radians
from datetime import datetime
from types import SimpleNamespace


import cadquery as cq
from jupyter_cadquery import show, set_defaults, open_viewer, Camera
from jupyter_cadquery.replay import enable_replay, disable_replay, reset_replay, get_context, replay, Replay, _CTX
from ocp_tessellate.convert import to_assembly

cv = open_viewer("Box", cad_width=780, height=525)
set_defaults(reset_camera=Camera.RESET, show_parent=False, axes=True, axes0=True)

use_replay = True
if use_replay:
    enable_replay(show_bbox=False, warning=False)
    reset_replay()
    show_object = replay
else:
    disable_replay()
    show_object = show

print(f"Initialized jupyter_cadquery with replay {datetime.now()}")
m = SimpleNamespace()
m.hook_length = 48	
m.beam_width = 16	
m.beam_height = 24	
m.beam_angle = 12.500	
m.drop = m.hook_length*sin(radians(m.beam_angle)) # positive
m.reach = m.hook_length*cos(radians(m.beam_angle))
m.body_thickness_far = 8
m.body_thickness_near = m.body_thickness_far + 0.5*m.drop
m.wing_thickness = 8
m.wing_drop_near = m.beam_height * 1.5
m.wing_drop_far = m.beam_height * 0.6667
m.wing_height_near = m.body_thickness_near + m.wing_drop_near
m.wing_height_far  = m.body_thickness_far + m.wing_drop_far
m.edge_fillet = 0.5


print(f"Building from model {m}")


Enabling jupyter_cadquery replay
Initialized jupyter_cadquery with replay 2025-11-16 15:57:52.083557
Building from model namespace(hook_length=48, beam_width=16, beam_height=24, beam_angle=12.5, drop=10.389101469028938, reach=46.8622083417568, body_thickness_far=8, body_thickness_near=13.194550734514468, wing_thickness=8, wing_drop_near=36.0, wing_drop_far=16.000799999999998, wing_height_near=49.194550734514465, wing_height_far=24.000799999999998, edge_fillet=0.5)


In [37]:
# Make the top body
s = (
        cq.Workplane("YZ")
        .moveTo(0,m.body_thickness_near)
        # New
        .polarLine(m.body_thickness_near, -90)
        .polarLine(m.hook_length, m.beam_angle)
        .polarLine(m.body_thickness_far, 90)
        .close()

)
# box1 = cq.Workplane("YZ").placeSketch(s)
right_body = s.extrude(m.beam_width/2)
replay(right_body)


Use the multi select box below to select one or more steps you want to examine
c

In [38]:
# Make a wing
right_wing = (
    right_body.faces(">X").workplane()
    .moveTo(0,m.body_thickness_near)
    .polarLine(m.wing_height_near, -90) # down
    .line(m.reach, m.wing_drop_near + m.drop - m.wing_drop_far) # diag to far
    .polarLine(m.wing_height_far, 90) # up
    .close()

).extrude(m.wing_thickness)

replay(right_wing)

Use the multi select box below to select one or more steps you want to examine
c

In [42]:
# Mirror in YZ plane
left_wing = right_wing.mirror(mirrorPlane="YZ", basePointVector=(0, 0, -30))
# body = left_wing + right_wing
body = right_wing.union(left_wing)



## Make the jaw
```
Select -X face of left wing
Select -Y,-Z vertex

Sketch jaw...
Extrude wing_thickness
Sketch bottom jaw...
Extrude wing_thickness
Make jaw extension (as solid?)
Make tooth
```

In [43]:
# Fillet the edges
body = body.edges().fillet(m.edge_fillet) 
replay(body)

Use the multi select box below to select one or more steps you want to examine
c